In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
key=os.getenv("openai_api_key")

In [5]:
llm=ChatOpenAI(openai_api_key=key, model_name='gpt-3.5-turbo', temperature=0.5)

C:\Users\ASIF\AppData\Local\Temp\ipykernel_8924\3734199289.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(openai_api_key=key, model_name='gpt-3.5-turbo', temperature=0.5)


In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

### Input Prompt Template

In [7]:
Response_Json={
    '1':{
        'mcq':'multiple choice question',
        'options':{
            'a':'choice here',
            'b':'choice here',
            'c':'choice here',
            'd':'choice here'
        },
        'correct':'correct answer',
    },
    '2':{
        'mcq':'multiple choice question',
        'options':{
            'a':'choice here',
            'b':'choice here',
            'c':'choice here',
            'd':'choice here'
        },
        'correct':'correct answer',
    },
    '3':{
        'mcq':'multiple choice question',
        'options':{
            'a':'choice here',
            'b':'choice here',
            'c':'choice here',
            'd':'choice here'
        },
        'correct':'correct answer',
    },
}

In [8]:
TEMPLATE="""
Text : {text}
You are an expert mcq maker. Given the above text, it is your job to \
create a quiz of {number} multiple choices questions for {subject} students in {tone} tone.
make sure the questions are not repeated and check all the questions to be conforming the text as well.
make sure to format your response like RESPONSE_JSON below and use it as guide. \
Ensure to make {number} mcqs.
### RESPONSE_JSON
{Response_Json}

"""

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text','number', 'subject', 'tone', 'Response_Json'],
    template=TEMPLATE
)

In [10]:
from env.Lib.tabnanny import verbose

quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key='quiz', verbose=True)

C:\Users\ASIF\AppData\Local\Temp\ipykernel_8924\2748869412.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key='quiz', verbose=True)


In [11]:
TEMPLATE2="""
you are an expert English grammarian and writer. Given a multiple choice quiz for {subject} students. \
you need to evaluate the complexity of questions and give a complete analysis of quiz.only use at max 50 words for complexity.
if the quiz is not as per with the cognitive and analytical abilities of the students. \
Update the quiz questions wich needs to be changed and change the tone such that it perfectly fits the students ability.
Quiz_MCQs.
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=['subject','quiz'],
    template=TEMPLATE2
)

In [13]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key='review', verbose=True)

In [14]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=['text','number', 'subject', 'tone', 'Response_Json'],
                                        output_variables=['quiz','review'], verbose=True)

In [15]:
file_path=r"C:\Users\ASIF\mcqgen\data.txt"

In [16]:
with open(file_path,'r') as file:
    Text=file.read()
    
print(Text)

Training Bidirectional Encoders (NSP)
 • An important class of applications involves determining the relationship between
 pairs of sentences– paraphrase detection (detecting if two sentences have similar meanings)– entailment (detecting if the meanings of two sentences entail or contradict each other)– discourse coherence (deciding if two neighboring sentences form a coherent discourse)
 • To capture the kind of knowledge required for applications such as these, BERT
 introduced a second learning objective called Next Sentence Prediction (NSP)
 • Training: The model is presented with pairs of sentences and is asked to predict
 whether each pair consists of an actual pair of adjacent sentences from the training
 corpus or a pair of unrelated sentences.
 • In BERT, 50% of the training pairs consisted of positive pairs, and in the other 50%
 the second sentence of a pair was randomly selected from elsewhere in the corpus– TheNSPloss is based on how well the model can distinguish true pai

In [17]:
### Serialize the py dict into a json-formatted string
json.dumps(Response_Json)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [19]:
Number  =5
Subject ='BERT'
Tone    ='Simple'

In [20]:
### How to setup token usage tracking in langchain

with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text"         :Text,
            "number"       :Number,
            "subject"      :Subject,
            "tone"         :Tone,
            "Response_Json":json.dumps(Response_Json)
        }
    )

C:\Users\ASIF\AppData\Local\Temp\ipykernel_8924\112805976.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text : Training Bidirectional Encoders (NSP)
 • An important class of applications involves determining the relationship between
 pairs of sentences– paraphrase detection (detecting if two sentences have similar meanings)– entailment (detecting if the meanings of two sentences entail or contradict each other)– discourse coherence (deciding if two neighboring sentences form a coherent discourse)
 • To capture the kind of knowledge required for applications such as these, BERT
 introduced a second learning objective called Next Sentence Prediction (NSP)
 • Training: The model is presented with pairs of sentences and is asked to predict
 whether each pair consists of an actual pair of adjacent sentences from the training
 corpus or a pair of unrelated sentences.
 • In BERT, 50% of the training pairs consisted of positive pairs, and in the other 50%
 the second sentence of a pair was rand

In [21]:
print('total tokens      : ',cb.total_tokens)
print('prompt tokens     :',cb.prompt_tokens)
print('completion tokens :',cb.completion_tokens)
print('total cost        :',cb.total_cost)

total tokens      :  1481
prompt tokens     : 1033
completion tokens : 448
total cost        : 0.0024455


In [22]:
response

{'text': 'Training Bidirectional Encoders (NSP)\n • An important class of applications involves determining the relationship between\n pairs of sentences– paraphrase detection (detecting if two sentences have similar meanings)– entailment (detecting if the meanings of two sentences entail or contradict each other)– discourse coherence (deciding if two neighboring sentences form a coherent discourse)\n • To capture the kind of knowledge required for applications such as these, BERT\n introduced a second learning objective called Next Sentence Prediction (NSP)\n • Training: The model is presented with pairs of sentences and is asked to predict\n whether each pair consists of an actual pair of adjacent sentences from the training\n corpus or a pair of unrelated sentences.\n • In BERT, 50% of the training pairs consisted of positive pairs, and in the other 50%\n the second sentence of a pair was randomly selected from elsewhere in the corpus– TheNSPloss is based on how well the model can d

In [23]:
quiz=response.get('quiz')

In [24]:
quiz=json.loads(quiz)

In [25]:
quiz_table_data=[]

for key, value in quiz.items():
    mcq=value['mcq']
    options=" | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value['options'].items()
        ]
    )
    correct=value['correct']
    quiz_table_data.append({"MCQ":mcq, "Choices":options, "correct_Ans":correct})

In [26]:
quiz_table_data

[{'MCQ': 'What is the purpose of the Next Sentence Prediction (NSP) task in BERT training?',
  'Choices': 'a: To predict the next word in a sentence | b: To determine if two sentences have similar meanings | c: To classify images in a dataset | d: To generate new text based on input',
  'correct_Ans': 'b'},
 {'MCQ': 'How is the NSP loss calculated in BERT training?',
  'Choices': "a: Based on the length of the input sentences | b: Based on the model's ability to distinguish true pairs from random pairs | c: Based on the number of layers in the model | d: Based on the learning rate of the optimizer",
  'correct_Ans': 'b'},
 {'MCQ': 'What tokens are introduced to the input representation in BERT for NSP?',
  'Choices': 'a: [CLS] and [SEP] | b: [START] and [END] | c: [TOKEN] and [MASK] | d: [BEGIN] and [STOP]',
  'correct_Ans': 'a'},
 {'MCQ': 'In BERT training, what does the output vector from the final layer associated with the [CLS] token represent?',
  'Choices': 'a: The first sentence

In [27]:
quiz_df=pd.DataFrame(quiz_table_data)

In [28]:
quiz_df.to_csv("BERT_Quiz.csv",index=False)

In [29]:
quiz_df

,MCQ,Choices,correct_Ans
0,What is the purpose of the Next Sentence Predi...,a: To predict the next word in a sentence | b:...,b
1,How is the NSP loss calculated in BERT training?,a: Based on the length of the input sentences ...,b
2,What tokens are introduced to the input repres...,a: [CLS] and [SEP] | b: [START] and [END] | c:...,a
3,"In BERT training, what does the output vector ...",a: The first sentence in the pair | b: The sec...,c
4,What percentage of training pairs in BERT cons...,a: 25% | b: 50% | c: 75% | d: 100%,b
